In [ ]:
#! pip install ibm-generative-ai 
#!pip install ibm-generative-ai[langchain] # IBM GenAI langchain interface
#! pip install langchain-experimental # for SQL Database Chain -- recently moved to -experimental module

from dotenv import load_dotenv, dotenv_values
from genai.credentials import Credentials
from genai.model import Model
from genai.schemas import GenerateParams
import os
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
try:
    from langchain import PromptTemplate
    from langchain.chains import LLMChain, SimpleSequentialChain
except ImportError:
    raise ImportError("Could not import langchain: Please install ibm-generative-ai[langchain] extension.")
from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams

from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate


In [ ]:
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url) # credentials object to access the LLM service


In [ ]:
# Instantiate parameters for text generation
params = GenerateParams(decoding_method="sample", max_new_tokens=10)

# Instantiate a model proxy object to send your requests
flan_ul2 = Model("google/flan-ul2", params=params, credentials=creds)

prompts = ["Hello! How are you?", "How's the weather?"]
for response in flan_ul2.generate(prompts):
    print(response.generated_text)


In [ ]:

print("\n------------- Example (Model Talk)-------------\n")

bob_params = GenerateParams(decoding_method="sample", max_new_tokens=25, temperature=1)
alice_params = GenerateParams(decoding_method="sample", max_new_tokens=45, temperature=0.5)
bob = Model("google/flan-ul2", params=bob_params, credentials=creds)
alice = Model("google/flan-t5-xxl", params=alice_params, credentials=creds)

sentence = "Hello! How are you?"

print(f"[Alice] --> {sentence}")
for i in range(10):
    bob_response = bob.generate([sentence])
    # from first batch get first result generated text
    bob_gen = bob_response[0].generated_text
    print(f"[Bob] --> {bob_gen}")

    alice_response = alice.generate([bob_gen])
    # from first batch get first result generated text
    alice_gen = alice_response[0].generated_text
    print(f"[Alice] --> {alice_gen}")

    sentence = alice_gen
    time.sleep(0.5)

In [ ]:
params = GenerateParams(
    decoding_method="sample",
    max_new_tokens=1,
    min_new_tokens=1,
    stream=False,
    temperature=0.7,
    top_k=50,
    top_p=1,
    return_options=ReturnOptions(input_text=False, input_tokens=True),
)

creds = Credentials(api_key, api_endpoint)
model = Model("google/flan-ul2", params=params, credentials=creds)

### Langchain Extension

In [ ]:

# make sure you have a .env file under genai root with

load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)


In [ ]:


params = GenerateParams(
    decoding_method="sample",
 #   decoding_method="greedy",
    max_new_tokens=1536,
    min_new_tokens=1,
    stream=False,
    temperature=0.05,
    top_k=50,
    top_p=1
).dict()  # Langchain uses dictionaries to pass kwargs

Models = ["google/flan-ul2","eleutherai/gpt-neox-20b","tiiuae/falcon-40b","google/flan-t5-xxl"]
m_idx = 0
LLM = LangChainInterface(model=Models[m_idx], credentials=creds, params=params,verbose=True)

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

template = """You are a helpful assistant who generates comma separated lists.
A user will pass in a category, and you should generate 5 objects in that category in a comma separated list.
ONLY return a comma separated list, and nothing more."""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(
#    llm=ChatOpenAI(),
    llm=LLM,
    prompt=chat_prompt,
    output_parser=CommaSeparatedListOutputParser()
)
chain.run("colors")

### DB Interface

In [ ]:


db = SQLDatabase.from_uri("sqlite:///chinook.db",include_tables=['Employee'],
    sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(LLM, db, verbose=False, use_query_checker=True, top_k = 5) 
db_chain.run("How many employees are there?")


In [ ]:
print(db.table_info)

In [ ]:

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = OpenAI(temperature=0, verbose=False,top_p=1)
db_chain_oai = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True, top_k = 5) 
db_chain_oai.run("How many employees are there?")

In [ ]:
db_chain.run("what are the firstname and lastname of the employees?")

In [ ]:
db_chain.run("what are the first name and last name of the employees?")

In [ ]:
db_chain_oai.run("what are the firstname and lastname of the employees?")

In [ ]:
db_chain_oai.run("what are the first name and last name of the employees?")

In [ ]:


_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct sql query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"
in table format

Only use the following tables:
CREATE TABLE [Album]
(
    [AlbumId] INTEGER  NOT NULL,
    [Title] NVARCHAR(160)  NOT NULL,
    [ArtistId] INTEGER  NOT NULL,
    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),
    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Artist]
(
    [ArtistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])
);

CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Employee]
(
    [EmployeeId] INTEGER  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [FirstName] NVARCHAR(20)  NOT NULL,
    [Title] NVARCHAR(30),
    [ReportsTo] INTEGER,
    [BirthDate] DATETIME,
    [HireDate] DATETIME,
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60),
    CONSTRAINT [PK_Employee] PRIMARY KEY  ([EmployeeId]),
    FOREIGN KEY ([ReportsTo]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Genre]
(
    [GenreId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Genre] PRIMARY KEY  ([GenreId])
);

CREATE TABLE [Invoice]
(
    [InvoiceId] INTEGER  NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Invoice] PRIMARY KEY  ([InvoiceId]),
    FOREIGN KEY ([CustomerId]) REFERENCES [Customer] ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [InvoiceLine]
(
    [InvoiceLineId] INTEGER  NOT NULL,
    [InvoiceId] INTEGER  NOT NULL,
    [TrackId] INTEGER  NOT NULL,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    [Quantity] INTEGER  NOT NULL,
    CONSTRAINT [PK_InvoiceLine] PRIMARY KEY  ([InvoiceLineId]),
    FOREIGN KEY ([InvoiceId]) REFERENCES [Invoice] ([InvoiceId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [MediaType]
(
    [MediaTypeId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_MediaType] PRIMARY KEY  ([MediaTypeId])
);

CREATE TABLE [Playlist]
(
    [PlaylistId] INTEGER  NOT NULL,
    [Name] NVARCHAR(120),
    CONSTRAINT [PK_Playlist] PRIMARY KEY  ([PlaylistId])
);

CREATE TABLE [PlaylistTrack]
(
    [PlaylistId] INTEGER  NOT NULL,
    [TrackId] INTEGER  NOT NULL,
    CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY  ([PlaylistId], [TrackId]),
    FOREIGN KEY ([PlaylistId]) REFERENCES [Playlist] ([PlaylistId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([TrackId]) REFERENCES [Track] ([TrackId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);

CREATE TABLE [Track]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId]),
    FOREIGN KEY ([AlbumId]) REFERENCES [Album] ([AlbumId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([GenreId]) REFERENCES [Genre] ([GenreId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY ([MediaTypeId]) REFERENCES [MediaType] ([MediaTypeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
);


If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
#    input_variables=["input", "table_info","dialect"], template=_DEFAULT_TEMPLATE
    input_variables=["input"], template=_DEFAULT_TEMPLATE
)
#PROMPT.format(table_info=['employee'], dialect='sql',input="How many employees are there?")

db_chain = SQLDatabaseChain.from_llm(LLM, db, prompt=PROMPT, verbose=True,use_query_checker=True)
db_chain.run("How many employees are there?")

In [ ]:
db_chain.run("what are the firstname and lastname of the employees")

In [ ]:
db_chain.run("what are the first name and last name of the employees")

In [ ]:
db_chain.run("return a list of employee names in markup format")

In [ ]:
db_chain_oai.run("return a list of employee names in markup format")

<ul><li>Andrew Adams</li><li>Nancy Edwards</li><li>Jane Peacock</li><li>Margaret Park</li><li>Steve Johnson</li></ul>'

In [ ]:
 db_chain_oai.run("List the total sales per country. Which country's customers spent the most?")

In [ ]:
db_chain.run("List the total sales per country. Which country's customers spent the most?")

In [ ]:
db_chain_oai.run("What are some example tracks by Bach?")

In [ ]:
db_chain.run("What are some example tracks by Bach?")